In [3]:
%load_ext blackcellmagic
%matplotlib inline

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [9]:
from matplotlib import style
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta as rd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Reflect Tables into SQLAlchemy ORM

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [10]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [11]:
# We can view all of the classes that automap found
Base.classes.keys()

In [12]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [13]:
# Create our session (link) from Python to the DB
session = Session(engine)
inspector = inspect(engine)
mCols = inspector.get_columns("measurement")
sCols = inspector.get_columns("station")


def colnames(col_list):
    for c in col_list:
        print(c["name"], c["type"])


print("Measurement columns:")
colnames(mCols)
print("Stations columns:")
colnames(sCols)

# Exploratory Climate Analysis

* Design a query to retrieve the last 12 months of precipitation data and plot the results

* Calculate the date 1 year ago from the last data point in the database

* Perform a query to retrieve the data and precipitation scores

* Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data

#### Design a query to retrieve the last 12 months of precipitation data and plot the results
#### Calculate the date 1 year ago from the last data point in the database

In [15]:
dates_uniq = []
for value in session.query(Measurement.date).distinct():
    dates_uniq.append(value.date)
end_date = max(dates_uniq)
start_date = str(dmax - dt.timedelta(365))[:10]
start_date

In [16]:
# Perform a query to retrieve the data and precipitation scores
df_prcp_data = []
df_prcp_sum = []

q = (
    session.query(Measurement.date, Measurement.prcp)
    .filter(Measurement.date >= start_date)
    .all()
)
for record in q:
    df_prcp_data.append(record)

In [17]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df_prcp = pd.DataFrame(df_prcp_data).set_index("date")
df_prcp = df_prcp.dropna(axis=0, how="any")
df_prcp

In [18]:
def plot_ticks(y, m, day, months):
    trip_data = [f"{y}-0{m}-{day}"]
    start_month = dt.datetime(y, m, day)
    for i in range(int(months / 2)):
        next_month = start_month + rd(months=+2)
        # next_month = next_month.strftime('%Y-%m-%d')
        # month_data = calc_temps(start_month, next_month)
        trip_data.append(next_month.strftime("%Y-%m-%d"))
        start_month = next_month
    return trip_data


x_ticks = plot_ticks(2016, 8, 23, 12)

In [189]:
x_ticks

['2016-08-23',
 '2016-10-23',
 '2016-12-23',
 '2017-02-23',
 '2017-04-23',
 '2017-06-23',
 '2017-08-23']

In [19]:
plt.scatter(x=df_prcp.index, y=df_prcp["prcp"])
plt.xticks(ticks=x_ticks, rotation=90)
plt.ylabel("Precipitation")
plt.title("Last 12 months of Precipitation data")
plt.savefig("plot_prcp.png")

#### Use Pandas to print the summary statistics for the precipitation data.

In [20]:
summ_dict = {
    "count": df_prcp.shape[0],
    "mean": df_prcp["prcp"].mean(),
    "std": df_prcp["prcp"].std(),
    "min": df_prcp["prcp"].min(),
    "25%": df_prcp["prcp"].quantile(0.25),
    "50%": df_prcp["prcp"].quantile(0.50),
    "75%": df_prcp["prcp"].quantile(0.75),
    "max": df_prcp["prcp"].max(),
}
df_summ = pd.DataFrame(summ_dict, index=("precipitation", 1))
df_summ = df_summ.drop([1]).T
df_summ

# Station Analysis

#### Design a query to show how many stations are available in this dataset?


In [21]:
stations = session.query(Station.station).distinct().all()
print(len(stations))
stations

#### List the stations and observation counts in descending order


In [22]:
df_data1 = []
df_data1_sums = []
query = (
    session.query(Measurement, func.count(Measurement.station))
    .group_by(Measurement.station)
    .all()
)
for record in query:
    df_data1.append(record[0].__dict__)
    df_data1_sums.append(record[1])

df_stations = pd.DataFrame(df_data1).set_index("station")
df_stations["counts"] = df_data1_sums
df_stations = df_stations.sort_values("counts", ascending=False,)
df_stations = pd.DataFrame(df_stations["counts"])
df_stations

#### Which station has the highest number of observations
##### USC00519281    Observations: 2772

#### Using the station id from the previous query, calculate the lowest temperature recorded, highest temperature recorded, and average temperature most active station?

In [23]:
q = (
    session.query(
        func.min(Measurement.tobs),
        func.avg(Measurement.tobs),
        func.max(Measurement.tobs),
    )
    .filter(Measurement.station == "USC00519281")
    .all()
)
print("Station: USC00519281", "min:", q[0][0], "max:", q[0][1], "avg:", q[0][2])

#### Choose the station with the highest number of temperature observations.
##### USC00519281    Observations: 2772
#### Query the last 12 months of temperature observation data for this station and plot the results as a histogram

In [24]:
# Most Active Station
# Measurement.station == USC00519281
df_tobs_data = []
q = session.query(Measurement.tobs).filter(Measurement.station == "USC00519281").all()
for record in q:
    df_tobs_data.append(record[0])

plt.xlabel("Temperature")
plt.ylabel("Observations")
plt.title("Last 12 Months of Temperature Data (USC00519281)")
fig = plt.hist(x=df_tobs_data, bins=12)
plt.savefig("plot_tobs.png")

## Optional Challenge Assignment

#### This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' and return the minimum, average, and maximum temperatures for that range of dates

In [223]:
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d        
    Returns:
        TMIN, TAVE, and TMAX
    """
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

#### Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax for your trip using the previous year's data for those same dates.

In [25]:
trip_start = "2016-04-01"
trip_end = "2016-04-11"
trip_early_start = str(
    dt.datetime.strptime(trip_start, "%Y-%m-%d") - dt.timedelta(365)
)[:10]
trip_early_end = str(dt.datetime.strptime(trip_end, "%Y-%m-%d") - dt.timedelta(365))[
    :10
]
plot_data = calc_temps(trip_start, trip_end)
plot_data_early = calc_temps(trip_early_start, trip_early_end)
print(plot_data, plot_data_early)

#### Plot the results from your previous query as a bar chart. Use "Trip Avg Temp" as your Title. Use the average temperature for the y value
#### Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

In [26]:
error = plot_data_early[0][0] - plot_data_early[0][2]
plt.title("Trip Avg Temp")
plt.ylabel("Temperature")
plt.grid(axis="x")
plt.bar(x=" ", height=plot_data_early[0][1], alpha=0.6)
plt.errorbar(x=0, y=plot_data_early[0][1], yerr=error, ecolor="black")
plt.savefig("plot_trip.png")

#### Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.

#### Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

In [27]:
df_data2 = []
df_data2_sums = []
sel = [
    Measurement.id,
    Measurement.prcp,
    Station.id,
    Station.station,
    Station.name,
    Station.latitude,
    Station.longitude,
    Station.elevation,
]
q = (
    session.query(*sel, func.sum(Measurement.prcp).label("Total_Prcp"))
    .filter(Measurement.station == Station.station)
    .filter(Measurement.date)
    .group_by("station")
    .all()
)
for record in q:
    df_data2.append(record)
df_prcp_sums = pd.DataFrame(df_data2)
df_prcp_sums.columns = [
    "id",
    "prcp",
    "fakeid",
    "station",
    "name",
    "latitude",
    "longitude",
    "elevation",
    "total_prcp",
]
df_prcp_sums = df_prcp_sums.drop(["fakeid", "prcp"], axis=1)
df_prcp_sums = df_prcp_sums.set_index("id").sort_values("total_prcp", ascending=False)
df_prcp_sums

In [20]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [21]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


[(62.0, 69.15384615384616, 77.0),
 (60.0, 69.39622641509433, 77.0),
 (62.0, 68.9090909090909, 77.0),
 (58.0, 70.0, 76.0),
 (56.0, 67.96428571428571, 76.0),
 (61.0, 68.96491228070175, 76.0),
 (57.0, 68.54385964912281, 76.0)]

In [22]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


,tmin,tavg,tmax
date,,,
2018-01-01,62.0,69.153846,77.0
2018-01-02,60.0,69.396226,77.0
2018-01-03,62.0,68.909091,77.0
2018-01-04,58.0,70.000000,76.0
2018-01-05,56.0,67.964286,76.0


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`